In [8]:
#!pip install yfinance

In [14]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
import pytz

def download_stock_data(symbols, timeframes, period="1mo"):
    all_data = []
    for timeframe in timeframes:
        data = yf.download(symbols, group_by='Ticker', period=period, interval=timeframe)
        data = data.stack(level=0).rename_axis(['Date', 'Ticker']).reset_index(level=1)
        data["Timeframe"] = timeframe
        data["Datetime"] = data.index
        
        # Convert 'Datetime' column to UTC
        data['Datetime'] = pd.to_datetime(data['Datetime'], utc=True)
        
        # Convert 'Datetime' column to 'Asia/Kolkata' time zone
        data['Datetime_India'] = data['Datetime'].dt.tz_convert(pytz.timezone('Asia/Kolkata'))
        
        # Split Datetime into Date, Time, and Timezone columns
        data['Date'] = data['Datetime_India'].dt.date
        data['Time'] = data['Datetime_India'].dt.time
        data['Timezone'] = data['Datetime_India'].dt.tz.zone
        # Add Camarilla levels for 1d timeframe
        if timeframe == "1d":
            data = calculate_camarilla_levels(data)
        
        all_data.append(data)
    return pd.concat(all_data, ignore_index=True)

def calculate_camarilla_levels(data):
    
    # Camarilla pivot levels
    
    data["H"] = data["High"]
    data["L"] = data["Low"]
    data["C"] = data["Close"]

    data["H1"] = data["C"] + 1.1 * (data["H"] - data["L"]) / 12
    data["H2"] = data["C"] + 1.1 * (data["H"] - data["L"]) / 6
    data["H3"] = data["C"] + 1.1 * (data["H"] - data["L"]) / 4
    data["H4"] = data["C"] + 1.1 * (data["H"] - data["L"]) / 2.66

    data["L1"] = data["C"] - 1.1 * (data["H"] - data["L"]) / 12
    data["L2"] = data["C"] - 1.1 * (data["H"] - data["L"]) / 6
    data["L3"] = data["C"] - 1.1 * (data["H"] - data["L"]) / 4
    data["L4"] = data["C"] - 1.1 * (data["H"] - data["L"]) / 2.66
    
    return data

def filter_data_by_date(data, days_ago):
    # Filters data for a specific date
    
    target_date = (datetime.now() - timedelta(days=days_ago)).date()
    return data[data['Date'] == target_date]

def save_to_csv(data, filepath):
    # Saves the given DataFrame to a CSV file.
    
    data.to_csv(filepath, index=False)
    print(f"Saved data to {filepath}")

symbols = ['VIPCLOTHNG.NS', 'HMVL.NS', 'LPDC.NS', 'GOENKA.NS']
timeframes = ["5m", "1d"]

# Historical data
combined_data = download_stock_data(symbols, timeframes)
save_to_csv(combined_data, "combined_equity_data.csv")

# Filter data for yesterday
filtered_data = filter_data_by_date(combined_data, days_ago=1)
save_to_csv(filtered_data, "filtered_equity_data_with_camarilla.csv")


[*********************100%***********************]  4 of 4 completed
/var/folders/8z/96_p5y5s313_8b55fgygvygh0000gn/T/ipykernel_66867/2030107288.py:10: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  data = data.stack(level=0).rename_axis(['Date', 'Ticker']).reset_index(level=1)
[*********************100%***********************]  4 of 4 completed
/var/folders/8z/96_p5y5s313_8b55fgygvygh0000gn/T/ipykernel_66867/2030107288.py:10: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  data = data.stack(level=0).rename_axis(['Date', 'Ticker']).reset_index(level=1)


Saved data to /Users/amitkumar/Documents/Screener/combined_equity_data.csv
Saved data to /Users/amitkumar/Documents/Screener/filtered_equity_data_with_camarilla.csv


In [ ]:
def fetch_latest_price(symbols):
    # Convert the list of symbols to a space-separated string
    symbols_str = " ".join(symbols)
    
    # Fetch ticker data for all symbols
    tickers = yf.Tickers(symbols_str)
    history_data = tickers.history(period="1d")
    
    # Extract closing prices for all symbols
    try:
        current_prices = history_data['Close'].iloc[-1].dropna()
        result_df = current_prices.reset_index()
        result_df.columns = ['Ticker', 'CurrentPrice']
    except Exception as e:
        print(f"Error fetching data: {e}")
        result_df = pd.DataFrame(columns=['Ticker', 'CurrentPrice'])
    
    return result_df

def camarilla_lower_tf_fill(group):
    # Filling the 5m camarilla data
    
    df_1d = group[group['Timeframe'] == '1d']
    df_5m = group[group['Timeframe'] == '5m']
    if not df_1d.empty:
        df_5m_filled = df_5m.copy()
        # Fill 5m with the latest available 1d values
        df_5m_filled[['H1','H2','H3','H4','L1','L2','L3','L4']] = df_1d[['H1','H2','H3','H4','L1','L2','L3','L4']].ffill().iloc[-1]
        # Combine 1d and filled 5m data
        return pd.concat([df_1d, df_5m_filled]).sort_index()
    else:
        return group

# Current Price 
current_price = fetch_latest_price(symbols)
latest_price = filtered_data.merge(current_price, on='Ticker', how='left')

# Applying group-by using Symbol and Date
camarilla_price_data = latest_price.groupby(['Ticker','Date'], group_keys=False).apply(camarilla_lower_tf_fill)

camarilla_price_data.to_csv("filtered_equity_data_with_camarilla.csv", index=False)

[*********************100%***********************]  4 of 4 completed
/var/folders/8z/96_p5y5s313_8b55fgygvygh0000gn/T/ipykernel_66867/3546231456.py:28: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df_5m_filled[['H1','H2','H3','H4','L1','L2','L3','L4']] = df_1d[['H1','H2','H3','H4','L1','L2','L3','L4']].ffill().iloc[-1]
/var/folders/8z/96_p5y5s313_8b55fgygvygh0000gn/T/ipykernel_66867/3546231456.py:28: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df_5m_filled[['H1','H2','H3','H4','L1','L2','L3','L4']] = df_1d[['H1','H2','H3','H4','L1','L2','L3','L4']].ffill().iloc[-1]
/var/folders/8z/96_p5y5s313_8b55fgygvygh0000gn/T/

In [ ]:
#### Logic for Intraday Long Position 

long_5m_high = camarilla_price_data[(camarilla_price_data['Timeframe'] == '5m') & (camarilla_price_data['Time'].astype(str) == '09:15:00')]
long_5m_high['H3'] = pd.to_numeric(long_5m_high['H3'], errors='coerce')
#print('long_5m_high',long_5m_high)

### H3 level price cross-over

long_5m = long_5m_high[(long_5m_high['Close'] >= long_5m_high['H3']) | (long_5m_high['CurrentPrice'] >= long_5m_high['H3'])] 
long_5m_a = long_5m['Ticker'].unique()

print('unique_close_long_H3_crossover:',long_5m_a)

long_5m.to_csv('unique_close_long_H3_crossover.csv', index=False)

### first candle high breakdown

long_5m_candle = long_5m_high.merge(camarilla_price_data, on='Ticker', how='inner')
#print('long_5m_candle',long_5m_candle)

long_5m_candle_breakdown = long_5m_candle[long_5m_candle['CurrentPrice_x'] >= long_5m_candle['High_x']]
long_5m_candle_breakdown_a = long_5m_candle_breakdown['Ticker'].unique()

print('unique_current_long_price_first_candle_crossover:',long_5m_candle_breakdown_a)

long_5m_candle_breakdown.to_csv('unique_current_long_price_first_candle_crossover.csv', index=False)

unique_close_long_H3_crossover: ['VIPCLOTHNG.NS']
unique_current_long_price_first_candle_crossover: ['HMVL.NS' 'LPDC.NS']


/var/folders/8z/96_p5y5s313_8b55fgygvygh0000gn/T/ipykernel_66867/4228054012.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  long_5m_high['H3'] = pd.to_numeric(long_5m_high['H3'], errors='coerce')


In [ ]:
##### Logic for Intraday Short Position

short_5m_low = camarilla_price_data[(camarilla_price_data['Timeframe'] == '5m') & (camarilla_price_data['Time'].astype(str) == '09:15:00')]
short_5m_low['L3'] = pd.to_numeric(short_5m_low['L3'], errors='coerce')

### L3 level price cross-over

short_5m = short_5m_low[(short_5m_low['Close'] <= short_5m_low['L3']) | (short_5m_low['CurrentPrice'] <= short_5m_low['L3'])] 
short_5m_a = short_5m['Ticker'].unique()

print('unique_close_H3_short_crossover:',short_5m_a)
short_5m.to_csv('unique_close_H3_short_crossover.csv', index=False)

short_5m_candle = short_5m_low.merge(camarilla_price_data, on='Ticker', how='inner')

### first candle low breakdown

short_5m_candle_breakdown = short_5m_candle[short_5m_candle['CurrentPrice_x'] <= short_5m_candle['Low_x']]
short_5m_candle_breakdown_a = short_5m_candle_breakdown['Ticker'].unique()

print('unique_current_price_first_candle_crossover:',short_5m_candle_breakdown_a)

short_5m_candle_breakdown.to_csv('unique_current_price_first_candle_crossover.csv', index=False)

unique_close_H3_short_crossover: ['LPDC.NS' 'VIPCLOTHNG.NS']
unique_current_price_first_candle_crossover: ['VIPCLOTHNG.NS']


/var/folders/8z/96_p5y5s313_8b55fgygvygh0000gn/T/ipykernel_66867/1399483077.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  short_5m_low['L3'] = pd.to_numeric(short_5m_low['L3'], errors='coerce')
